#### INGEST / LOAD `ORDERS.JSON` INTO `BRONZE_ORDERS`

In [0]:
CREATE OR REFRESH STREAMING TABLE BRONZE_ORDERS
COMMENT 'Ingesting raw orders data as JSON files and loading into bronze table'
TBLPROPERTIES('quality' = 'bronze')
AS
SELECT 
*,
_metadata.file_path AS file_path,
CURRENT_TIMESTAMP AS ingestion_timestamp
FROM 
cloud_files('/Volumes/circuitbox_dev/landing/operational_data/orders/',
'json',
map('cloudFiles.inferColumnTypes', 'true')
)

#### PERFORM DATA QUALITY CHECKS

In [0]:
CREATE STREAMING TABLE SILVER_ORDERS
(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card', 'Bank Transfer', 'PayPal')),
  CONSTRAINT valid_order_status EXPECT (order_status IN ('Pending', 'Completed', 'Shipped', 'Cancelled'))
)
COMMENT 'Perform Data Quality Checks'
TBLPROPERTIES('quality' = 'silver')
AS
SELECT
customer_id,
order_id,
order_status,
items,
CAST(order_timestamp AS timestamp) AS order_timestamp,
payment_method
FROM STREAM(LIVE.BRONZE_ORDERS)

In [0]:
CREATE OR REFRESH STREAMING TABLE GOLD_ORDERS
AS
SELECT
customer_id,
order_id,
order_status,
order_timestamp,
payment_method,
item.name::STRING AS name,
item.price::DOUBLE AS price,
item.quantity::INT AS quantity,
item.category::STRING AS category,
item.item_id::INT AS item_id
FROM
(
SELECT
customer_id,
order_id,
explode(items) AS item,
order_status,
order_timestamp,
payment_method
FROM
STREAM(LIVE.SILVER_ORDERS));